# 4. Saving image data transmitted from the smartphone as image files

In this tutorial, use iOS application **intdash Motion** to stream video data, and save the data as images. In this case, we will focus on the following methods.

* Retrieve M-JPEG data saved on the server as binary data
* Save data as image files (.jpg)

## 4.0 Preparation
Before starting this scenario, prepare the following.

- An edge that performs measurement
- A measurement uploaded with intdash Motion (which contains images)


### Data to be used
In this scenario, the following data needs to be ready on the server side.

| Data item | Data name that appears in this scenario |
|:---|:---|
| Edge to register time series data | edge1|
|Measurement(*) |Measurement name does not appear |

(\*) Register time series data in the same way as **"intdash tutorial 2A: Visualizing smartphone data in real time"** using intdash Motion.

### Create measurement data and upload time series data
Upload the data using **intdash Motion**. After registering the measurement, use **Visual M2M Data Visualizer** to check that the new measurement has been created.

For uploading method, refer to **intdash tutorial 2A: Visualizing smartphone data in real time**.

Confirm that the new measurement is displayed in [Stored Data] of Data Visualizer.

<img src="https://github.com/aptpod/aws-marketplace-tutorials/blob/master/img/img4.png?raw=true\">

### Import packages and create a client
For `url` given to `intdash.Client`, specify the environment of the intdash server. For `username` and `password`, specify the auth information issued for the edge you use.

In [1]:
import pandas as pd

import intdash
from intdash import timeutils

# Create client
client = intdash.Client(
    url = "https://example.intdash.jp",
    username = "edge1",
    password="password_here"
)

The preparations are complete.

## 4.1 Retrieve the edge used for measurement
Retrieve the edge used when logging in with intdash Motion.

In [2]:
edges = client.edges.list(name='edge1')
edge1 = edges[0]

In [3]:
edge1.name

'edge1'

## 4.2 Retrieve time series data
In this scenario, the `client.data_points` endpoint is used to retrieve the time series data. Normally, specify ``labels`` to retrieve data using signal definitions, but in this case, use `id_queries` because signal definitions are not used.  
``start`` and ``end`` should be changed to a time that includes the measurements created in the previous step.

* Normally, a ``data_id`` needs to be specified. But for JPEG data, it is not necessary to specify the ``data_id`` because the ``data_id`` for JPEG is fixed. For details, see [intdash.data.Jpeg](https://docs.intdash.jp/sdk/python/latest/en/data.html#intdash.data.JPEG) in the document.

In [4]:
dps = client.data_points.list(
    edge_name='edge1',
    start=timeutils.str2timestamp('2020-07-17 00:00:00+09:00'),  # change appropriately.
    end=timeutils.str2timestamp('2020-07-18 00:00:00+09:00'),  # change appropriately.
    id_queries=[
        intdash.IdQuery(
            data_type=intdash.DataType.jpeg,
            channel=1
        )
    ]
)

## 4.3 Save the retrieved data as image files
Save the retrieved binary data as it is as JPEG.
Here, as a sample, only the first data point of the retrieved time series data is saved.

In [5]:
data = dps[0] #  Get the first data.

with open(f'./{pd.Timestamp(data.time).value}.jpg', 'wb') as fout:
    # Use the timestamp as the filename.
    fout.write(data.data_payload)

If you want to save all time series data, repeat the above.

In [6]:
import os 

save_dir='./images'

for d in dps:
    
    if not os.path.isdir(save_dir):
            os.makedirs(save_dir)
            
    # Use the timestamp as the filename.
    with open(f'{save_dir}/{pd.Timestamp(d.time).value}.jpg', 'wb') as fout:
        fout.write(d.data_payload)